WRF_Chem_Preppy - Nov. 2023

Preprocessor to obtain:
- Biogenic emissions
- Anthropogenic emissions
- Burning emissions 

CO2 and CH4 over our study area.

In [1]:
import datetime
import subprocess
import pandas as pd
#from cdo import *
import warnings

ModuleNotFoundError: No module named 'pandas'

IMPORTANT DATA:

- domains --> Number of domains.
- wrf_inp --> Path where are the "wrfinputs" files.
- wrf_geo --> Path where are the "geo_em" files.
- sim_time --> It is necessary to set the start and end of the processing.

In [3]:
domains        = 3                                                 # check this always
wrf_inp        = '../input/wrfinput/wrfinput_d0'   # check this always
wrf_geos        = '../input/domains/geo_em.d0'   # check this always

sim_time       = '2022-08-01 00:00:00','2022-08-31 23:00:00'       # check this!!
dates          = pd.to_datetime(sim_time[0]).strftime('%Y-%m-%d')

pos            = '.nc'  
projection_wrf = 'Lambert Conformal'

1. Preparing the data for the Kaplan model (biogenic emissions) -> CH4

  There are three python scripts to obtain the necessary inputs for the Kaplan model and calculate biogenic methane emissions.
  - prep_wetland_kaplan.py
  - prep_cpool_lpj.py
  - prep_T_ann.py

In [3]:
#These ones are updated 2023-11 ---> Noelia Rojas B.
for i,j in enumerate(range(domains),start=1):
    dom              = i
    wrf_inp_p        = wrf_inp  + str(i)
    wrf_geo_p        = wrf_geos  + str(i) + pos

    # ================= Kaplan inventory
    wet_var          = 'wetland'
    path_wet         = '../../../DATA/sources/VPRM/ch4_bio_input/global/global_wetland_kaplan.nc'
    wet_out          = '../../../DATA/sources/VPRM/ch4_bio_input/regrid/'
    lat              = 'lat'
    lon              = 'lon'
    regrid_method    = 'conservative'
    source           = 'Kaplan'
    
    #print(datetime.datetime.now())
    #%run -i prep_wetland_kaplan.py
    #print(datetime.datetime.now())
    
    cpool_var        = 'cpool_fast'
    cpool_path       = '../../../DATA/sources/VPRM/ch4_bio_input/global/lpj_cpool_2000.nc'
    cpool_out        = '../../../DATA/sources/VPRM/ch4_bio_input/regrid/'
    cmmd             = 'cdo remapbil,'+ wrf_inp + ' -fillmiss2 ' + cpool_path + ' intermediate_cpool_regrid.nc'
    
    #print(datetime.datetime.now())
    #subprocess.call(cmmd,shell=True)
    #%run -i prep_cpool_lpj.py
    #print(datetime.datetime.now()) 
       
    # Remember to check the year. The netcdf files for temp should be a Monthly mean. shape = (12).
    ### If changing year, remeber to change T_ANN year down below
    t_ann_path       = '../../../DATA/sources/VPRM/ch4_bio_input/global/ERA5_monthly_surface_temperatures_2022.nc'
    t_var            = 'stl1' # remember to include the other options
    lat              = 'latitude'
    lon              = 'longitude'
    t_ann_out        = '../../../DATA/sources/VPRM/ch4_bio_input/regrid/'
    regrid_method    = 'conservative'
    
    #print(datetime.datetime.now())
    #subprocess.call(cmmd,shell=True)
    #%run -i prep_T_ann.py
    #print(datetime.datetime.now())
    
    #!rm intermediate_cpool_regrid.nc
    
    print('===================================')
    print('FINISHED WITH DOMAIN %s'%(str(dom)))
    print('===================================')    
    

FINISHED WITH DOMAIN 1
FINISHED WITH DOMAIN 2
FINISHED WITH DOMAIN 3
FINISHED WITH DOMAIN 4


Juntando las emisiones biogenicas del CO2 y CH4 en un archivo VPRM_input

In [6]:
for i,j in enumerate(range(domains),start=1):
    
    dom              = i
    wrf_inp_p        = wrf_inp  + str(i)
    wrf_geo_p        = wrf_geos  + str(i) + pos

    ## join CH4 and CO2 biogenic dataset
    
    ######### READ CH4 BIO -- Kaplan inventory
    cpoolp  = cpool_out +'CPOOL_d0%s_%s.nc'%(dom,pd.to_datetime(dates).year)
    tannp   = t_ann_out + 'T_ANN_d0%s_%s.nc'%(dom,pd.to_datetime(dates).year)
    wetmapp = wet_out + 'WETMAP_d0%s_%s.nc'%(dom,pd.to_datetime(dates).year)

    ######### READ CO2 BIO ---- prec vprm    
    vprm_inp  = '../../../DATA/sources/VPRM/co2_bio_input/regrid/'
    evi_minp  = vprm_inp + 'd0%s/VPRM_input_EVI_MIN_%s.nc'%(dom,pd.to_datetime(dates).year)
    evi_maxp  = vprm_inp + 'd0%s/VPRM_input_EVI_MAX_%s.nc'%(dom,pd.to_datetime(dates).year)
    evip      = vprm_inp + 'd0%s/VPRM_input_EVI_%s.nc'%(dom,pd.to_datetime(dates).year)
    lswi_minp = vprm_inp + 'd0%s/VPRM_input_LSWI_MIN_%s.nc'%(dom,pd.to_datetime(dates).year)
    lswi_maxp = vprm_inp + 'd0%s/VPRM_input_LSWI_MAX_%s.nc'%(dom,pd.to_datetime(dates).year)
    lswip     = vprm_inp + 'd0%s/VPRM_input_LSWI_%s.nc'%(dom,pd.to_datetime(dates).year)
    vegfrap   = vprm_inp + 'd0%s/VPRM_input_VEG_FRA_%s.nc'%(dom,pd.to_datetime(dates).year)
        
    file_out = '../../../DATA/sources/VPRM/vprm_input/'
    #print(datetime.now())
    %run -i Join_vprm_input.py
    #print(datetime.now())   
    
    print('===================================')
    print('FINISHED WITH DOMAIN %s'%(str(dom)))
    print('===================================')    

generating netcdf file..
DONE GENERATING THE VPRM INPUT
FINISHED WITH DOMAIN 1
generating netcdf file..
DONE GENERATING THE VPRM INPUT
FINISHED WITH DOMAIN 2
generating netcdf file..
DONE GENERATING THE VPRM INPUT
FINISHED WITH DOMAIN 3
generating netcdf file..
DONE GENERATING THE VPRM INPUT
FINISHED WITH DOMAIN 4


Preparando los datos del GFAS para los cuatro dominios

In [ ]:
for i,j in enumerate(range(domains),start=1):
    
    dom              = i
    wrf_inp          = w + pre  + str(i)
    wrf_geo_p        = w_geo + pre2 + str(i) + pos

    # Remember to check the year. The netcdf files for temp should be a.....
    ### If changing year, remeber to change GFAS for each time 
    gfas_path       = '../../../DATA/sources/FIRES/global/2208_ghg.nc'
    var             = ['co2fire','ch4fire','cofire'] # remember to include the other options
    mvar             = [28.01,44.01,16.043]
    lat             = 'latitude'
    lon             = 'longitude'
    gfas_out        = '../../../DATA/sources/FIRES/regrid/'
    regrid_method   = 'conservative'
    
    print(datetime.datetime.now())
    %run -i pre_gfas.py
    print(datetime.datetime.now())
    
    #!rm intermediate_cpool_regrid.nc
    
    print('===================================')
    print('FINISHED WITH DOMAIN %s'%(str(dom)))
    print('===================================')    

Preparando los datos del EDGAR y WETCHART para los cuatro dominios

In [ ]:
for i,j in enumerate(range(domains),start=3):
    dom              = i
    wrf_inp          = w + pre  + str(i)
    wrf_geo_p        = w_geo + pre2 + str(i) + pos

    # Remember to check the year. The netcdf files for temp should be a.....
    ### If changing year, remeber to change GFAS for each time 
    edgar_path      = '../../../DATA/sources/ANTHRO/global/EDGAR/'
    wchts_path      = '../../../DATA/sources/ANTHRO/global/wetchart/'
    var             = ['CO','CO2','CH4'] # remember to include the other options
    mvar            = [28.01,44.01,16.043]
    num_model       = ['2913','2923','2933','2914','2924','2934']    #### for wetcharts data
    month           = int(pd.to_datetime(sim_time[0]).strftime('%m'))
    lat             = 'lat'
    lon             = 'lon'
    edgar_out       = '../../../DATA/sources/ANTHRO/regrid/'
    regrid_method   = 'conservative'
    
    print(datetime.datetime.now())
    %run -i prep_edgar.py
    print(datetime.datetime.now())
    
    #!rm intermediate_cpool_regrid.nc
    
    print('===================================')
    print('FINISHED WITH DOMAIN %s'%(str(dom)))
    print('===================================')  

In [ ]:
for i,j in enumerate(range(domains),start=3):
    dom              = i
    wrf_inp          = w + pre  + str(i)
    wrf_geo_p        = w_geo + pre2 + str(i) + pos